# Тестовое для СДЕК летней стажировки

Задача:
Предсказать цену (price) на аренду жилья, используя не только табличные данные, но и координаты.

Сслыка: https://www.kaggle.com/datasets/dgomonov/new-york-city-airbnb-open-data/data

In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("data/AB_NYC_2019.csv")

rows, cols = df.shape

print(f"Total rows: {rows}")
print(f"Total columns: {cols}")

Total rows: 48895
Total columns: 16


In [3]:
print(df.columns)

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')


Колонки для анализа:
- host_id - уникальные id владельца (заменим host_name)

In [ ]:
host_id = df["host_id"]

ids, cnts = np.unique(host_id, return_counts=True)
sorter = np.argsort(ids)
ids = ids[sorter]
cnts = cnts[sorter]

274321313
